In [ ]:
# get connection_string
import os
connection_string = os.getenv("CONN_STRING")

In [ ]:
! pip install azure-storage-blob

In [ ]:
# import azure blob storage SDK & create blob client to test connectivity
from azure.storage.blob import BlobClient
my_blob = BlobClient.from_connection_string(conn_str=connection_string,container_name="campaign0", blob_name="G0061732.JPG")
print("blob URL:",my_blob.url)
print("blob properties:", my_blob.get_blob_properties())

In [ ]:
# import container client and enumerate blob within container
from azure.storage.blob import ContainerClient

container = ContainerClient.from_connection_string(conn_str=connection_string, container_name="campaign0")

blob_list = container.list_blobs()
for blob in blob_list:
    print(blob.name)

In [ ]:
# pre-requesite: install pyodbc slq driver for python
! pip install pyodbc

In [ ]:
# import sql odbc driver and connect to SQL DB
import pyodbc
server = os.getenv("SERVER")
database = os.getenv("DATABASE")
username = os.getenv("USERNAME")
password = os.getenv("PWD")
driver= '{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:
# Sample Query on dbo.Images
cursor.execute("SELECT * FROM dbo.Images")
row = cursor.fetchone()
while row:
    print (str(row[0]))
    row = cursor.fetchone()

In [ ]:
# Get ColName & DataType of dbo.Images
cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME ='Images' ORDER BY ORDINAL_POSITION")
row = cursor.fetchone()
col_list = []
while row:
    print (str(row[0]))
    col_list.append(str(row[0]))
    row = cursor.fetchone()

# create a string of Table columns list
def colAsString(column_list):
    col_as_string= column_list[0]
    for col in col_list[1:len(column_list)]:
        col_as_string = col_as_string + ',' + col
    return col_as_string
colAsString(col_list)

In [ ]:
# SELECT COUNT 
cursor.execute("SELECT COUNT(*) FROM dbo.Images ")
row = cursor.fetchone()
while row:
    print (str(row[0]))
    row = cursor.fetchone()

In [ ]:
# SELECT Query on dbo.Images
cursor.execute("SELECT Id,Filename,BlobName,ContainerUrl,CreatedBy,CreatedOn,Tag,IsDeleted,Latitude,Longitude FROM dbo.Images")
row = cursor.fetchone()
while row:
    for col in row:
        print (str(col))
    row = cursor.fetchone()

In [ ]:
# INSERT Query on dbo.Images - First Example
cursor.execute("INSERT dbo.Images (Id, Filename, BlobName, ContainerUrl, CreatedBy, CreatedOn, Tag, IsDeleted, Latitude, Longitude ) VALUES (NEWID(),'test', 'test', 'test', 'test','2020-01-24' , 'bottle', 'FALSE','50.797731', '2.179029')")
cnxn.commit()

In [ ]:
# Create a list of blob names
blob_list = container.list_blobs()
blob_names_list = []
for blob in blob_list:
    blob_names_list.append(blob.name)
print(blob_names_list)
print(len(blob_names_list))

# Example: Extract Blob URL, Properties & Keys for blob0
blob0 = BlobClient.from_connection_string(conn_str=connection_string,container_name="campaign0", blob_name=blob_names_list[0])
blob0_name = blob_names_list[0]
blob0_url = blob0.url
blob0_prop = blob0.get_blob_properties()
blob0_prop_keys = blob0_prop.keys()
print("blob name:",blob0_name)
print("blob URL:",blob0_url)
print("blob properties:", blob0_prop)
print("blob properties keys:", blob0_prop_keys)

# INSERT Sample Images

In [ ]:
# INSERT no parameter
cursor.execute("INSERT dbo.Images (Id, Filename, BlobName, ContainerUrl, CreatedBy, CreatedOn, Tag, IsDeleted, Latitude, Longitude ) VALUES (NEWID(),'test', 'test', 'test', 'test','2020-01-24' , 'bottle', 'FALSE','50.797731', '2.179029')")
cnxn.commit()

In [ ]:
# INSERT parameterized
cursor.execute("INSERT dbo.Images (Id, Filename, BlobName, ContainerUrl, CreatedBy, CreatedOn, Tag, IsDeleted, Latitude, Longitude ) VALUES (NEWID(),?, ?, ?, 'test','2020-01-24' , 'bottle', 'FALSE','50.797731', '2.179029')",blob0_name,blob0_name,blob0_url)
cnxn.commit()

In [ ]:
# SELECT Query on dbo.Images
cursor.execute("SELECT Id,Filename,BlobName,ContainerUrl,CreatedBy,CreatedOn,Tag,IsDeleted,Latitude,Longitude FROM dbo.Images")
row = cursor.fetchone()

while row:
    print(str(row[0]))
    print(str(row[1]))
    print(str(row[3]+"\n"))
    row = cursor.fetchone()

# INSERT Images Loop over all Blobs in Campaign0 container

In [ ]:
# Get Blob names & url in container Campaign0
def getBlobNamesUrl(blob_names):
    for i in range(0,len(blob_names)):
        blob_i = BlobClient.from_connection_string(conn_str=connection_string,container_name="campaign0", blob_name=blob_names[i])
        blob_i_name = blob_names[i]
        blob_i_url = blob_i.url
        blob_i_prop = blob_i.get_blob_properties()
        blob_i_prop_keys = blob_i_prop.keys()
        print(blob_i_name)
        print(blob_i_url)
getBlobNamesUrl(blob_names_list)

In [ ]:
# Check Table content
cursor.execute("SELECT Id,Filename,BlobName,ContainerUrl,CreatedBy,CreatedOn,Tag,IsDeleted,Latitude,Longitude FROM dbo.Images")
row = cursor.fetchone()
while row:
    print(str(row[0]))
    row = cursor.fetchone()

In [ ]:
# INSERT parameterized
for i in range(0,len(blob_names_list)):
    blob_i = BlobClient.from_connection_string(conn_str=connection_string,container_name="campaign0", blob_name=blob_names_list[i])
    blob_i_name = blob_names_list[i]
    blob_i_url = blob_i.url
    blob_i_prop = blob_i.get_blob_properties()
    blob_i_prop_keys = blob_i_prop.keys()
    cursor.execute("INSERT dbo.Images (Id, Filename, BlobName, ContainerUrl, CreatedBy, CreatedOn, Tag, IsDeleted, Latitude, Longitude ) VALUES (NEWID(),?, ?, ?, 'test','2020-01-24' , 'bottle', 'FALSE','50.797731', '2.179029')",blob_i_name,blob_i_name,blob_i_url)
cnxn.commit()

In [ ]:
# COUNT rows in table after insert
cursor.execute("SELECT COUNT (Id) FROM dbo.Images ")
row = cursor.fetchone()
while row:
    print (str(row[0]))
    row = cursor.fetchone()

In [ ]:
cursor.execute("SELECT Id,Filename,BlobName,ContainerUrl,CreatedBy,CreatedOn,Tag,IsDeleted,Latitude,Longitude FROM dbo.Images")
row = cursor.fetchone()
while row:
    print(str(row[0]))
    print(str(row[1]))
    print(str(row[3]))
    print(str(row[6])+"\n")
    row = cursor.fetchone()

# Clean Images Table

In [ ]:
# DELETE ALL RECORD
cursor.execute("DELETE FROM dbo.Images")
cnxn.commit()